In [21]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain.tools import tool
import os

load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")


llm = ChatOpenAI(
    model="deepseek/deepseek-v3.2",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    temperature=0.0
)

### Часть 1: Базовые инструменты — атомарные операции

In [22]:
@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 2

### Часть 2: Создание субагентов

Каждый субагент — это полноценный агент, но с узкой специализацией:
- subagent_1 умеет только извлекать корень,
- subagent_2 умеет только возводить в квадрат.
- У каждого ограниченный набор инструментов → соблюдается принцип минимальных привилегий.


✅ Преимущества:

- Изолированность: ошибка в одном субагенте не ломает другой,
- Независимое развитие: можно заменить subagent_1 на другой без трогания subagent_2,
- Лучшая observability: в LangSmith видно, какой субагент что делал.

In [23]:
subagent_1 = create_agent(
    model=llm,
    tools=[square_root]
)

subagent_2 = create_agent(
    model=llm,
    tools=[square]
)

### Часть 3: Обёртка субагентов в инструменты


##### Как это работает?
- Основной агент решает: «мне нужно извлечь корень из 456».
- Он вызывает инструмент call_subagent_1(456).
- Инструмент внутри себя запускает subagent_1.invoke(...).
- subagent_1:
    - Получает сообщение "Calculate the square root of 456",
    - Понимает, что нужно вызвать square_root(456),
    - Выполняет вычисление → получает 21.354...,
    - Возвращает результат в виде AIMessage.
- Инструмент call_subagent_1 извлекает .content из последнего сообщения и возвращает число (на самом деле — строку, см. ниже).
- Основной агент получает этот результат и формирует финальный ответ.

In [24]:
@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calcualte the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of{x}")]})
    return response["messages"][-1].content

def call_subagent_2(x: float) -> float:
    """Call subagent 1 in order to calcualte the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

main_agent = create_agent(
    model=llm,
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number."
)

In [25]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [26]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='dea5498e-e1c8-49e4-81b8-cbd1e12d012f'),
              AIMessage(content="I'll calculate the square root of 456 for you using the appropriate tool.\n\n", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 152, 'total_tokens': 179, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0}, 'cost': 5.005e-05, 'is_byok': False, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 3.952e-05, 'upstream_inference_completions_cost': 1.053e-05}}, 'model_provider': 'openai', 'model_name': 'deepseek/deepseek-v3.2', 'system_fingerprint': None, 'id': 'gen-1767467875-Ju5pBkzBIKmeFamllK3W', 'finish_reas